# **Project 2**
Seung Min Song, Krutika Patel<br>

03/05/2024

# **Data Source**

* https://www.kaggle.com/datasets/sarmadriaz/a-network-analysis-of-game-of-thrones?rvi=1

In [261]:

import pandas as pd

# Load dataset

got_df = pd.read_csv('https://raw.githubusercontent.com/seung-m1nsong/620/main/Project2/GoT_book1.csv')

got_df.head()

,Source,Target,Type,weight,book
0,Addam-Marbrand,Jaime-Lannister,Undirected,3,1
1,Addam-Marbrand,Tywin-Lannister,Undirected,6,1
2,Aegon-I-Targaryen,Daenerys-Targaryen,Undirected,5,1
3,Aegon-I-Targaryen,Eddard-Stark,Undirected,4,1
4,Aemon-Targaryen-(Maester-Aemon),Alliser-Thorne,Undirected,4,1


In [239]:
got_df = got_df[['Source', 'Target', 'weight']]
got_df.head()


,Source,Target,weight
0,Addam-Marbrand,Jaime-Lannister,3
1,Addam-Marbrand,Tywin-Lannister,6
2,Aegon-I-Targaryen,Daenerys-Targaryen,5
3,Aegon-I-Targaryen,Eddard-Stark,4
4,Aemon-Targaryen-(Maester-Aemon),Alliser-Thorne,4


In [240]:
got_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 684 entries, 0 to 683
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Source  684 non-null    object
 1   Target  684 non-null    object
 2   weight  684 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 16.2+ KB


In [241]:
node_sizes = got_df.groupby('Source').weight.agg(sum)
n = node_sizes.add_suffix('').reset_index()
top_15 = n.sort_values(by='weight', ascending=False).head(15)
print(top_15)

                 Source  weight
42         Eddard-Stark    1013
18           Bran-Stark     511
22        Catelyn-Stark     475
9            Arya-Stark     430
32   Daenerys-Targaryen     420
24     Cersei-Lannister     367
79             Jon-Snow     335
76    Joffrey-Baratheon     275
69      Jaime-Lannister     157
13      Barristan-Selmy     143
124    Robert-Baratheon     136
40                Drogo     135
122          Robb-Stark     120
72         Jeor-Mormont     119
110       Petyr-Baelish     118


In [237]:
import networkx as nx

G = nx.from_pandas_edgelist(got_df, 'Source', 'Target', ['weight'])
print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())
print("Average degree:", sum(dict(G.degree()).values()) / G.number_of_nodes())


Number of nodes: 187
Number of edges: 684
Average degree: 7.315508021390374


## Grouped Visualization

In [260]:
from itertools import combinations
from pyvis.network import Network

net = Network(notebook=True)

# Define node groups
group_stark = [n for n in G.nodes() if 'Stark' in n]
group_targaryen = [n for n in G.nodes() if 'Targaryen' in n]
group_lannister = [n for n in G.nodes() if 'Lannister' in n]
group_jon_snow = [n for n in G.nodes() if 'Jon-Snow' in n]
group_baratheon = [n for n in G.nodes() if 'Baratheon' in n]

for n in G.nodes():
    group = None
    if n in group_stark:
        group = 'Stark'
    elif n in group_targaryen:
        group = 'Targaryen'
    elif n in group_lannister:
        group = 'Lannister'
    elif n in group_jon_snow:
        group = 'Jon-Snow'
    elif n in group_baratheon:
        group = 'Baratheon'
    else:
        group = 'Other'  # You can define additional groups if needed

    color = 'green' if group == 'Stark' else 'orange' if group == 'Targaryen' else 'blue' if group == 'Lannister' else 'red' if group == 'Jon-Snow' else 'yellow' if group == 'Baratheon' else 'gray'
    size = 10 + 5 * G.degree(n)  # adjust the multiplier for desired size scaling
    net.add_node(n, label=n, color=color, size=size, group=group)

# Add more edges within each group for cohesion
for group in [group_stark, group_targaryen, group_lannister, group_jon_snow, group_baratheon]:
    for node1, node2 in combinations(group, 2):
        if not any(edge['from'] == node1 and edge['to'] == node2 for edge in net.edges):
            net.add_edge(node1, node2)


# apply barnes_hut algorithm for better node separation
net.barnes_hut()

# show buttons for filtering nodes
net.show_buttons(filter_=['nodes'])

# save the graph as an HTML file
net.save_graph('grouped.html')


In [244]:
from itertools import combinations, product
from pyvis.network import Network

net = Network(notebook=True)

# Define node groups
group_stark = [n for n in G.nodes() if 'Stark' in n]
group_targaryen = [n for n in G.nodes() if 'Targaryen' in n]
group_lannister = [n for n in G.nodes() if 'Lannister' in n]
group_jon_snow = [n for n in G.nodes() if 'Jon-Snow' in n]
group_baratheon = [n for n in G.nodes() if 'Baratheon' in n]

for n in G.nodes():
    group = None
    if n in group_stark:
        group = 'Stark'
    elif n in group_targaryen:
        group = 'Targaryen'
    elif n in group_lannister:
        group = 'Lannister'
    elif n in group_jon_snow:
        group = 'Jon-Snow'
    elif n in group_baratheon:
        group = 'Baratheon'
    else:
        group = 'Other'  # You can define additional groups if needed

    color = 'green' if group == 'Stark' else 'orange' if group == 'Targaryen' else 'blue' if group == 'Lannister' else 'red' if group == 'Jon-Snow' else 'yellow' if group == 'Baratheon' else 'gray'
    size = 10 + 5 * G.degree(n)  # adjust the multiplier for desired size scaling
    net.add_node(n, label=n, color=color, size=size, group=group)

# Add more edges within each group for cohesion
for group in [group_stark, group_targaryen, group_lannister, group_jon_snow, group_baratheon]:
    for node1, node2 in combinations(group, 2):
        if not any(edge['from'] == node1 and edge['to'] == node2 for edge in net.edges):
            net.add_edge(node1, node2)

# Add edges between nodes in different groups
for node1, node2 in product(G.nodes(), G.nodes()):
    if node1 != node2:
        net.add_edge(node1, node2)


# apply barnes_hut algorithm for better node separation
net.barnes_hut()

# show buttons for filtering nodes
net.show_buttons(filter_=['nodes'])

# save the graph as an HTML file
net.save_graph('bbbba.html')


## Source & Target Visualization

In [245]:
from pyvis.network import Network

net = Network(notebook=True)

for n in G.nodes():
    # Check if the node exists in the 'Source' or 'Target' column of got_df
    if n in got_df['Source'].tolist():
        color = 'green'
    elif n in got_df['Target'].tolist():
        color = 'orange'
    else:
        # Skip nodes that do not exist in got_df
        continue

    # set size based on degree (number of edges)
    size = 10 + 5 * G.degree(n)  # adjust the multiplier for desired size scaling
    net.add_node(n, label=n, color=color, size=size)

# add edges
for edge in G.edges():
    if edge[0] in net.get_nodes() and edge[1] in net.get_nodes():
        net.add_edge(edge[0], edge[1])

# apply barnes_hut algorithm for better node separation
net.barnes_hut()

# show buttons for filtering nodes
net.show_buttons(filter_=['nodes'])

# save the graph as an HTML file
net.save_graph('SourceTarget.html')

## Color Grouped Visualization


In [248]:
from pyvis.network import Network

net = Network(notebook=True)

# Define node groups
group_stark = [n for n in G.nodes() if 'Stark' in n]
group_targaryen = [n for n in G.nodes() if 'Targaryen' in n]
group_lannister = [n for n in G.nodes() if 'Lannister' in n]
group_jon_snow = [n for n in G.nodes() if 'Jon-Snow' in n]
group_baratheon = [n for n in G.nodes() if 'Baratheon' in n]

for n in G.nodes():
    group = None
    if n in group_stark:
        group = 'Stark'
    elif n in group_targaryen:
        group = 'Targaryen'
    elif n in group_lannister:
        group = 'Lannister'
    elif n in group_jon_snow:
        group = 'Jon-Snow'
    elif n in group_baratheon:
        group = 'Baratheon'    
    else:
        group = 'Other' 

    color = 'green' if group == 'Stark' else 'orange' if group == 'Targaryen' else 'blue' if group == 'Lannister' else 'red' if group == 'Jon-Snow' else 'yellow' if group == 'Baratheon'  else 'blue'
    size = 10 + 5 * G.degree(n)  # adjust the multiplier for desired size scaling
    net.add_node(n, label=n, color=color, size=size, group=group)

# add edges
for edge in G.edges():
    if edge[0] in net.get_nodes() and edge[1] in net.get_nodes():
        net.add_edge(edge[0], edge[1])

# apply barnes_hut algorithm for better node separation
net.barnes_hut()

# show buttons for filtering nodes
net.show_buttons(filter_=['nodes'])

# save the graph as an HTML file
net.save_graph('groupColored.html')


In [127]:
# A function that outputs 10 neighbors in the order of highest weight between specific nodes (names) in network G

def find_ten_closest(G, name):  
    neighbors_list = list(G.neighbors(f"{name}"))
    
    temp_dict = {}
    for n in neighbors_list:
        if G.has_edge(f"{name}", n):
            weight = G.edges[f"{name}", n]['weight']
            temp_dict[n] = weight
    
    sorted_list = sorted(temp_dict.items(), key=(lambda x:x[1]), reverse=True)
    for n, w in sorted_list[0:10]:  # 1~10위
        print(f'{n}:  {w}')

# Output the top 10 people closest to 'Jon-Snow' and their weight
find_ten_closest(G, 'Jon-Snow')

Jeor-Mormont:  81
Samwell-Tarly:  81
Bran-Stark:  56
Tyrion-Lannister:  56
Robb-Stark:  53
Pypar:  45
Benjen-Stark:  41
Eddard-Stark:  38
Arya-Stark:  37
Aemon-Targaryen-(Maester-Aemon):  34
